In [2]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 72.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.6 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3


In [3]:
from huggingface_hub import login
login()

In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.5 MB/s eta 0:00:00:00:0100:01


In [5]:
import os
import faiss
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from typing import List
import textwrap

class HFLLMAdapter:
    def __init__(self, model_id="/kaggle/input/llama_law_tn_v3/keras/default/6", device=None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if "cuda" in self.device else torch.float32,
            device_map="auto"
        )

    def generate(self, prompt: str, max_new_tokens: int = 256) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=0.5,
                do_sample=False
            )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)


2025-12-08 00:58:24.459439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765155504.685907      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765155504.752624      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [6]:
import re
import ast
import textwrap
import torch
from typing import List, Tuple


LEGAL_PROMPT = """\
Bạn là **trợ lý pháp lý chuyên nghiệp**, chuyên tư vấn và phân tích **theo pháp luật Việt Nam**.

❗ Yêu cầu bắt buộc:
- Chỉ được chọn **1 đáp án đúng nhất**: A, B, C hoặc D.
- Trình bày ngắn gọn, không được sinh 4 mục.
- Nội dung bắt buộc gồm 2 phần:
    1) `Đáp án đúng: A/B/C/D`
    2) Một câu lập luận ngắn gọn (tối đa 1 câu) giải thích tại sao chọn đáp án đó.

🔎 Định dạng trả về (bắt buộc):
`Đáp án đúng: A/B/C/D – [lập luận 1 câu]`

---

**Câu hỏi trắc nghiệm:**
{query}

**Các căn cứ pháp lý được truy xuất từ hệ thống RAG:**
{context_text}

---

➡️ **Phần trả lời chính thức:**
"""


class LegalRAGProcessor:
    def __init__(self, llm_adapter, retriever, top_k=2, max_new_tokens=256):
        self.llm = llm_adapter
        self.retriever = retriever
        self.top_k = top_k
        self.max_new_tokens = max_new_tokens

    def _build_prompt(self, query, context_text):
        return LEGAL_PROMPT.format(query=query.strip(), context_text=context_text.strip())

    def _clean_context(self, docs: List[Tuple[str, float]]) -> str:
        merged = []
        seen = set()
        for i, (text, score) in enumerate(docs, 1):
            cleaned = re.sub(r"[ \t]+", " ", text.strip())
            cleaned = cleaned.replace("Ï", "I")
            if cleaned not in seen:
                seen.add(cleaned)
                merged.append(f"[ĐOẠN {i}] {cleaned}")
        return "\n\n".join(merged[: self.top_k])

    def _generate(self, prompt_text: str) -> str:
        inputs = self.llm.tokenizer(prompt_text, return_tensors="pt", truncation=True).to(self.llm.device)
        outputs = self.llm.model.generate(
            **inputs,
            max_new_tokens=self.max_new_tokens,
            temperature=0.1,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=False,
            eos_token_id=self.llm.tokenizer.eos_token_id,
        )
        text = self.llm.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return self._clean_response(text)

    # ==========================================================
    # CLEAN-UP: Trích đáp án + tạo lập luận ngắn (1 câu)
    # ==========================================================
    def _clean_response(self, text: str) -> str:
        # Tách phần sau “Phần trả lời chính thức”
        text = re.sub(r"(?is)^.*?Phần trả lời chính thức[:：]\s*", "", text).strip()

        # -------------------------
        # 1) Trích ra đáp án A/B/C/D
        # -------------------------
        patterns = [
            r"[Đđ]áp án đúng[:：]?\s*([ABCD])",
            r"[Cc]họn[:：]?\s*([ABCD])",
            r"\b([ABCD])\b",
        ]
        answer = None
        for p in patterns:
            m = re.search(p, text)
            if m:
                answer = m.group(1).upper()
                break

        if answer is None:
            answer = "Không có câu trả lời đúng"

        # -------------------------
        # 2) Lấy câu lập luận ngắn (1 câu)
        # -------------------------
        text_no_ans = re.sub(r"[Đđ]áp án đúng[:：]?\s*[ABCD]", "", text)
        text_no_ans = text_no_ans.replace("*", "").strip()

        sentences = re.split(r"[.!?。]", text_no_ans)
        reason = ""

        for s in sentences:
            s_clean = s.strip()
            if len(s_clean) > 100:
                reason = s_clean
                break

        reason = reason.strip()

        return f"Đáp án đúng: {answer} – {reason}"

    # ---------------------------
    # API chính
    # ---------------------------
    def answer(self, query: str):
        docs = self.retriever.retrieve(query, top_k=self.top_k)
        context_text = self._clean_context(docs)
        prompt = self._build_prompt(query, context_text)
        return self._generate(prompt)


In [7]:
!pip install rank_bm25

In [8]:
import re
import numpy as np
import faiss
from rank_bm25 import BM25Okapi
from typing import List, Tuple, Dict
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


class HybridLegalRetriever:
    def __init__(
        self,
        embedder_path: str = "bkai-foundation-models/vietnamese-bi-encoder",
        reranker_model: str = "cross-encoder/ms-marco-MiniLM-L-12-v2",
        top_k: int = 2,
        batch_size: int = 8,
    ):
        # Embedding model
        self.embedder = SentenceTransformer(embedder_path)
        self.top_k = top_k
        self.batch_size = batch_size

        # Reranker (sequence classification model)
        self.reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_model)
        self.reranker_model = AutoModelForSequenceClassification.from_pretrained(reranker_model)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.reranker_model.to(self.device)
        self.reranker_model.eval()

        # Indexes / storage
        self.bm25 = None
        self.faiss_index = None
        self.text_chunks: List[str] = []
        self.embeddings = None

    # =========================================================
    # Build index: BM25 + FAISS
    # =========================================================
    def build_index(self, chunks: List[str]):
        if not chunks:
            raise ValueError("❌ Danh sách chunks rỗng, không thể xây dựng index.")

        self.text_chunks = chunks

        # --- BM25 index (tokenize with words) ---
        print("🔹 Xây dựng BM25 index...")
        tokenized_corpus = [re.findall(r'\w+', c.lower()) for c in chunks]
        self.bm25 = BM25Okapi(tokenized_corpus)

        # --- FAISS index (embedding vectors) ---
        print("🔹 Đang tạo embeddings và xây dựng FAISS index...")
        embeddings = self.embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
        embeddings = np.asarray(embeddings, dtype=np.float32)

        # normalize and add to index
        faiss.normalize_L2(embeddings)
        dim = embeddings.shape[1]
        self.faiss_index = faiss.IndexFlatIP(dim)
        self.faiss_index.add(embeddings)
        self.embeddings = embeddings

        print(f"✅ Hoàn tất! Đã xây dựng index cho {len(chunks):,} đoạn luật với vector dim = {dim}.")
        print(f"   • FAISS vectors: {self.faiss_index.ntotal}")
        print(f"   • Thiết bị reranker: {self.device}")

    # =========================================================
    # Topic priority (placeholder, can extend)
    # =========================================================
    #def _apply_topic_priority(self, query: str, text: str) -> float:
        # Example: boost when query contains keywords; keep 1.0 by default
    #    q = query.lower()
    #    if "mất năng lực" in q or "giám hộ" in q or "giao dịch" in q:
    #        # small boost for chunks that explicitly mention Điều/Khoản related
    #        if re.search(r"Điều\s+\d+", text):
    #            return 1.15
    #    return 1.0

    # =========================================================
    # Cross-encoder rerank. Returns list of (text, score)
    # =========================================================
    def _rerank(self, query: str, candidates: List[Tuple[str, float]]) -> List[Tuple[str, float]]:
        if not candidates:
            return []

        pairs = [(query, c[0]) for c in candidates]
        scores = []

        for batch_start in range(0, len(pairs), self.batch_size):
            batch = pairs[batch_start: batch_start + self.batch_size]
            # tokenizer accepts list of tuples for some HF tokenizers; for safety, join with special sep
            # we use encoding of pairs as two sentences
            enc = self.reranker_tokenizer(
                [q for q, _ in batch],
                [c for _, c in batch],
                padding=True,
                truncation=True,
                return_tensors="pt",
                max_length=512,
            )
            enc = {k: v.to(self.device) for k, v in enc.items()}
            with torch.no_grad():
                logits = self.reranker_model(**enc).logits  # shape (batch, num_labels)
                # If num_labels >1, pick first logit or compute score appropriately.
                # Many cross-encoders return single logit per pair (regression); handle both cases:
                if logits.dim() == 2 and logits.size(1) == 1:
                    batch_scores = torch.sigmoid(logits).squeeze(-1).cpu().numpy()
                else:
                    # fallback: take softmax over labels and take max-prob
                    probs = torch.softmax(logits, dim=1)
                    batch_scores = probs[:, 0].cpu().numpy()  # heuristic
            scores.extend(batch_scores.tolist())

        reranked = [(c[0], float(s)) for c, s in zip(candidates, scores)]
        reranked.sort(key=lambda x: x[1], reverse=True)
        return reranked

    # =========================================================
    # Retrieve top_k candidates
    # =========================================================
    def retrieve(self, query: str, top_k: int = None) -> List[Tuple[str, float]]:
        if self.faiss_index is None or self.bm25 is None:
            raise RuntimeError("❌ Chưa có index. Hãy gọi build_index() trước khi truy vấn.")

        if top_k is None:
            top_k = self.top_k

        # --- FAISS semantic search ---
        q_emb = self.embedder.encode([query], convert_to_numpy=True)
        q_emb = np.asarray(q_emb, dtype=np.float32)
        faiss.normalize_L2(q_emb)
        D, I = self.faiss_index.search(q_emb, top_k)
        faiss_results = []
        for idx, score in zip(I[0], D[0]):
            if idx < 0:
                continue
            faiss_results.append((self.text_chunks[int(idx)], float(score)))

        # --- BM25 lexical search ---
        bm25_scores = self.bm25.get_scores(re.findall(r'\w+', query.lower()))
        bm25_top_idx = np.argsort(bm25_scores)[::-1][:top_k]
        bm25_results = [(self.text_chunks[int(i)], float(bm25_scores[int(i)])) for i in bm25_top_idx]

        # --- Merge, keep best score per doc ---
        merged: Dict[str, float] = {}
        for text, score in faiss_results + bm25_results:
            merged[text] = max(merged.get(text, 0.0), float(score))

        # --- Apply topic boosting ---
        #boosted = [(text, score * self._apply_topic_priority(query, text)) for text, score in merged.items()]

        # --- Rerank by cross-encoder ---
        merged_list = [(text, score) for text, score in merged.items()]
        reranked = self._rerank(query, merged_list)

        # --- Final top_k ---
        results = reranked[:top_k]

        # Print brief preview for debugging
        print(f"🔎 Truy vấn: {query!s}")
        print(f"📑 Top {len(results)} kết quả:")
        for i, (text, score) in enumerate(results, 1):
            preview = text.replace("\n", " ")[:200]
            print(f"   {i:02d}. ({score:.4f}) {preview}...")

        return results

    def __repr__(self):
        n = len(self.text_chunks)
        dim = self.embeddings.shape[1] if self.embeddings is not None else "?"
        return f"<HybridLegalRetriever: {n} docs | dim={dim} | device={self.device}>"


In [9]:
import re
import os
from tqdm import tqdm
from typing import List

def normalize_legal_text(text: str) -> str:
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"(?<=Điều)\s+(\d+)", r" \1.", text)
    text = re.sub(r"(?<=Khoản)\s+(\d+)", r" \1.", text)
    text = re.sub(r"(?<!\.)\s*(?=Điều\s+\d+\.)", "\n", text)
    text = re.sub(r"(?<!\.)\s*(?=Khoản\s+\d+\.)", "\n", text)
    return text.strip()

def _tag_structures(text: str) -> str:
    """Gắn tag [CHƯƠNG], [ĐIỀU], [KHOẢN]"""
    chapter = re.findall(r"(CHƯƠNG\s+[IVXLC]+[^Đ]*)", text)
    article = re.findall(r"(ĐIỀU LUẬT\s+\d+)", text)
    clause = re.findall(r"(KHOẢN\s+\d+)", text)

    ch = f"[CHƯƠNG] {chapter[0].strip()}" if chapter else ""
    di = f"[ĐIỀU LUẬT] {article[0].strip()}" if article else ""
    kh = f"[KHOẢN] {clause[0].strip()}" if clause else ""

    return " ".join([ch, di, kh]).strip()

def split_legal_text(text: str, max_length: int = 1000) -> List[str]:
    """Tách văn bản luật và gắn nhãn cấu trúc."""
    text = re.sub(r"\s+", " ", text)
    sections = re.split(r"(?=Điều\s+\d+\.?)", text)
    refined_chunks = []

    for sec in sections:
        if len(sec.strip()) < 20:
            continue
        if len(sec) > max_length * 2:
            subchunks = re.split(r"(?=Khoản\s+\d+\.?)", sec)
            refined_chunks += [f"{_tag_structures(s)} {s.strip()}" for s in subchunks if len(s.strip()) > 50]
        else:
            refined_chunks.append(f"{_tag_structures(sec)} {sec.strip()}")

    return [c.strip() for c in refined_chunks if len(c.strip()) > 50]



def extract_text_from_txt(txt_path: str) -> str:
    """Đọc nội dung văn bản từ file .txt (UTF-8)."""
    try:
        with open(txt_path, "r", encoding="utf-8") as f:
            text = f.read()
        return text
    except UnicodeDecodeError:
        # fallback nếu file mã hóa khác
        with open(txt_path, "r", encoding="utf-8-sig", errors="ignore") as f:
            return f.read()


def build_rag_from_txt_folder(folder_path: str):
    """
    Đọc tất cả file .txt trong thư mục, chia nhỏ thành các đoạn (chunks)
    để dùng làm dữ liệu cho mô hình RAG.
    """
    txt_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".txt")]
    print(f"📚 Đang đọc {len(txt_files)} tệp TXT từ: {folder_path}")

    all_chunks = []
    for file_name in tqdm(txt_files):
        txt_path = os.path.join(folder_path, file_name)
        try:
            text = extract_text_from_txt(txt_path)
            # ✅ Dùng hàm mới
            chunks = split_legal_text(text, max_length=1000)
            all_chunks.extend(chunks)
        except Exception as e:
            print(f"⚠️ Lỗi khi xử lý {file_name}: {e}")

    print(f"✅ Tổng số đoạn văn bản (chunks): {len(all_chunks)}")
    return all_chunks


In [10]:
from tqdm import tqdm
txt_folder = "/kaggle/input/luat-txt" 
llm = HFLLMAdapter("/kaggle/input/llama_law_tn_v3/keras/default/6")
retriever = HybridLegalRetriever()
docs = build_rag_from_txt_folder(txt_folder)
retriever.build_index(docs)

rag = LegalRAGProcessor(llm, retriever)

question = "Theo Bộ luật Dân sự, cá nhân có quyền có họ, tên (bao gồm cả chữ đệm, nếu có). Vậy, họ của cá nhân được xác định như thế nào trong trường hợp cha mẹ không có thỏa thuận? \nA. Theo họ của cha đẻ. \nB. Theo họ của cha đẻ hoặc họ của mẹ đẻ theo thỏa thuận của cha mẹ; nếu không có thỏa thuận thì họ của con được xác định theo tập quán. \nC. Theo họ của mẹ đẻ. \nD. Theo thỏa thuận của cha mẹ, hoặc nếu không có thỏa thuận thì theo quyết định của cơ quan nhà nước có thẩm quyền."
answer = rag.answer(question)
print("\n======================")
print("❓ Câu hỏi:", question)
print("💬 Trả lời:", answer)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

...

100%|██████████| 3/3 [00:00<00:00, 30.98it/s]

✅ Tổng số đoạn văn bản (chunks): 1228
🔹 Xây dựng BM25 index...
🔹 Đang tạo embeddings và xây dựng FAISS index...


Batches:   0%|          | 0/39 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


✅ Hoàn tất! Đã xây dựng index cho 1,228 đoạn luật với vector dim = 768.
   • FAISS vectors: 1228
   • Thiết bị reranker: cuda
🔎 Truy vấn: Theo Bộ luật Dân sự, cá nhân có quyền có họ, tên (bao gồm cả chữ đệm, nếu có). Vậy, họ của cá nhân được xác định như thế nào trong trường hợp cha mẹ không có thỏa thuận? 
A. Theo họ của cha đẻ. 
B. Theo họ của cha đẻ hoặc họ của mẹ đẻ theo thỏa thuận của cha mẹ; nếu không có thỏa thuận thì họ của con được xác định theo tập quán. 
C. Theo họ của mẹ đẻ. 
D. Theo thỏa thuận của cha mẹ, hoặc nếu không có thỏa thuận thì theo quyết định của cơ quan nhà nước có thẩm quyền.
📑 Top 2 kết quả:
   01. (0.8936) Điều 26. Quyền có họ, tên [KHOẢN] 1. Cá nhân có quyền có họ, tên (bao gồm cả chữ đệm, nếu có). Họ, tên của một người được xác định theo họ, tên khai sinh của người đó. [KHOẢN] 2. Họ của cá nhân được x...
   02. (0.2967) Điều 27. Quyền thay đối họ [KHOẢN] 1. Cá nhân có quyền yêu cầu cơ quan nhà nước có thâm quyển công nhận việc thay đôi họ trong trường hợp 

In [11]:
import re
import json
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import os, json, time, gc, torch
from tqdm import tqdm

def extract_choice(text: str):
    """Lấy lựa chọn A/B/C/D từ câu trả lời."""
    if not text or not isinstance(text, str):
        return None
    m = re.search(r"\b([ABCD])\b", text.upper())
    return m.group(1) if m else None


def evaluate_multiple_choice(references, predictions):
    """Trả về metrics và confusion matrix (dạng pandas DataFrame)."""

    true_labels = []
    pred_labels = []

    for ref, pred in zip(references, predictions):
        true_labels.append(extract_choice(ref) or "X")
        pred_labels.append(extract_choice(pred) or "X")

    labels = ["A", "B", "C", "D"]

    accuracy = accuracy_score(true_labels, pred_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels, pred_labels, labels=labels, average="weighted", zero_division=0
    )

    metrics = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1": f1
    }

    cm = confusion_matrix(true_labels, pred_labels, labels=labels)
    cm_df = pd.DataFrame(
        cm,
        index=[f"True {l}" for l in labels],
        columns=[f"Pred {l}" for l in labels]
    )

    return metrics, cm_df


def run_rag_pipeline(json_folder, rag_processor, output_dir="results_json"):
    """
    Chạy LegalRAGProcessor trên tập JSON trắc nghiệm:
    [
      {"id": ..., "question": ..., "answer": ..., "choice": ..., "context": ..., "pdf": ...}
    ]
    """
    os.makedirs(output_dir, exist_ok=True)

    all_refs, all_preds, results = [], [], []
    gen_times = []   # 🔥 Lưu thời gian sinh mỗi câu
    json_files = [f for f in os.listdir(json_folder) if f.endswith(".json")]

    for filename in tqdm(json_files, desc="⚖️ Đang chạy LegalRAGProcessor"):
        filepath = os.path.join(json_folder, filename)

        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)

        if isinstance(data, list):
            entries = data
        elif isinstance(data, dict):
            entries = [data]
        else:
            print(f"⚠️ Bỏ qua {filename} vì định dạng không hợp lệ.")
            continue

        for idx, item in enumerate(entries):
            question = item.get("question", "")
            reference = item.get("choice", "")     # 🔥 MCQ: dùng "choice"
            context = item.get("context", "")
            pdf_file = item.get("pdf", "")

            if not question:
                print(f"⚠️ Bỏ qua item {idx} trong {filename} vì thiếu 'question'.")
                continue

            try:
                start = time.time()
                answer = rag_processor.answer(question)
                elapsed = round(time.time() - start, 2)

                # 🔥 Lưu thời gian sinh
                gen_times.append(elapsed)

                if not answer:
                    print(f"⚠️ Không sinh được câu trả lời cho câu {idx} trong {filename}")
                    continue

                print("\n" + "=" * 80)
                print(f"📄 File: {filename} — Câu {idx+1}")
                print(f"📘 Câu hỏi: {question}")
                print(f"💬 Model Answer: {answer}")
                print(f"⏱️ Thời gian sinh: {elapsed}s")
                print("=" * 80)

                # Lưu để đánh giá
                all_refs.append(reference)   # 🔥 Mốc đúng: choice A/B/C/D
                all_preds.append(answer)     # 🔥 Model answer

                sample_result = {
                    "id": item.get("id", None),
                    "file": filename,
                    "question": question,
                    "choice": reference,
                    "context": context,
                    "model_answer": answer,
                    "gen_time": elapsed
                }
                results.append(sample_result)

                # Lưu từng câu riêng
                base = os.path.splitext(filename)[0]
                sample_out_path = os.path.join(
                    output_dir, f"{base}_q{idx+1}_result.json"
                )
                with open(sample_out_path, "w", encoding="utf-8") as fw:
                    json.dump(sample_result, fw, ensure_ascii=False, indent=2)

                torch.cuda.empty_cache()
                gc.collect()

            except Exception as e:
                print(f"❌ Lỗi khi xử lý {filename} (câu {idx}): {repr(e)}")
                continue

    # ===============================
    # 🔥 ĐÁNH GIÁ MCQ
    # ===============================
    if not all_refs:
        print("⚠️ Không có kết quả để đánh giá.")
        return [], [], 0

    metrics, cm_df = evaluate_multiple_choice(all_refs, all_preds)

    # 🔥 Tính thời gian sinh trung bình
    avg_time = sum(gen_times) / len(gen_times) if gen_times else 0

    print(f"\n⏱️ Thời gian sinh trung bình mỗi câu: {avg_time:.2f}s")

    print("\n🎯 KẾT QUẢ TRUNG BÌNH:")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

    print("\n🔢 CONFUSION MATRIX:")
    print(cm_df)

    # Lưu kết quả tổng hợp
    out_path = os.path.join(output_dir, "rag_pipeline_results.json")
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(
            {
                "results": results,
                "overall_metrics": metrics,
                "avg_gen_time": avg_time
            },
            f,
            ensure_ascii=False,
            indent=2
        )

    cm_df.to_csv(os.path.join(output_dir, "confusion_matrix.csv"), encoding="utf-8-sig")

    print(f"\n📁 File tổng hợp được lưu tại: {out_path}")

    return metrics, results, avg_time


In [12]:
json_folder = "/kaggle/input/luat-benchmark-tn/Benchmark_QA"
preds, results, avg_time = run_rag_pipeline(json_folder, rag)

⚖️ Đang chạy LegalRAGProcessor:   0%|          | 0/9 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


🔎 Truy vấn: Người sử dụng lao động có quyền tạm đình chỉ công việc của người lao động trong thời hạn tối đa là bao lâu trong trường hợp bình thường? 
A. Không quá 07 ngày. 
B. Không quá 15 ngày. 
C. Không quá 30 ngày. 
D. Không quá 90 ngày.
📑 Top 2 kết quả:
   01. (0.9993) Điều 128. Tạm đình chỉ công việc [KHOẢN] 1. Người sử dụng lao động có quyền tạm đình chỉ công việc của người lao động khi vụ việc vi phạm có những tình tiết phức tạp nếu xét thấy để người lao động tiế...
   02. (0.9967) Điều 31. Nhận lại người lao động hết thời hạn tạm hoãn thực hiện hợp đồng lao động | Trong thời hạn 15 ngày kế từ ngày hết thời hạn tạm hoãn thực hiện hợp đồng lao động, người lao động phải có mặt tại...

📄 File: 71-80.json — Câu 1
📘 Câu hỏi: Người sử dụng lao động có quyền tạm đình chỉ công việc của người lao động trong thời hạn tối đa là bao lâu trong trường hợp bình thường? 
A. Không quá 07 ngày. 
B. Không quá 15 ngày. 
C. Không quá 30 ngày. 
D. Không quá 90 ngày.
💬 Model Answer: Đáp án đúng: B – T

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


🔎 Truy vấn: Hợp đồng lao động có thời hạn dưới 01 tháng có thể được giao kết bằng hình thức nào? 
A. Bằng văn bản có công chứng, chứng thực. 
B. Bằng lời nói. 
C. Bằng văn bản có chữ ký của hai bên. 
D. Bằng văn bản và có hiệu lực ngay.
📑 Top 2 kết quả:
   01. (0.9985) Điều 14. Hình thức hợp đồng lao động [KHOẢN] 1. Hợp đồng lao động phải được giao kết bằng văn bản và được làm thành 02 bản, người lao động giữ 01 bản, người sử dụng lao động giữ 01 bản, trừ trường hợp...
   02. (0.9946) Điều 20. Loại hợp đồng lao động [KHOẢN] 1. Hợp đồng lao động phải được giao kết theo một trong các loại sau đây: a) Hợp đồng lao động không xác định thời hạn là hợp đồng mà trong đó hai bên không xác ...

📄 File: 71-80.json — Câu 2
📘 Câu hỏi: Hợp đồng lao động có thời hạn dưới 01 tháng có thể được giao kết bằng hình thức nào? 
A. Bằng văn bản có công chứng, chứng thực. 
B. Bằng lời nói. 
C. Bằng văn bản có chữ ký của hai bên. 
D. Bằng văn bản và có hiệu lực ngay.
💬 Model Answer: Đáp án đúng: B – Theo Điều

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

🔎 Truy vấn: Trong trường hợp nào, Tòa án không được từ chối giải quyết vụ, việc dân sự và phải áp dụng quy định về áp dụng tập quán và tương tự pháp luật? 
A. Vụ việc liên quan đến tài sản thừa kế. 
B. Vụ việc liên quan đến tranh chấp hợp đồng mua bán. 
C. Vì lý do chưa có điều luật để áp dụng. 
D. Vụ việc liên quan đến trách nhiệm bồi thường thiệt hại ngoài hợp đồng.
📑 Top 2 kết quả:
   01. (0.8994) Điều 14. Bảo vệ quyền dân sự thông qua cơ quan có thầm quyền [KHOẢN] 1. Tòa án, cơ quan có thâm quyền khác có trách nhiệm tôn trọng, bảo vệ quyền dân sự của cá nhân, pháp nhân. Trường hợp quyền dân sự...
   02. (0.8240) Điều 155. Không áp dụng thời hiệu khởi kiện Thời hiệu khởi kiện không áp dụng trong trường hợp sau đây: [KHOẢN] 1. Yêu cầu bảo vệ quyền nhân thân không gắn với tài sản; [KHOẢN] 2. Yêu cầu bảo vệ quyền...

📄 File: 21-30.json — Câu 9
📘 Câu hỏi: Trong trường hợp nào, Tòa án không được từ chối giải quyết vụ, việc dân sự và phải áp dụng quy định về áp dụng tập quán và tương tự p

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


🔎 Truy vấn: Theo Bộ luật Dân sự, vật phụ là gì? 
A. Là vật độc lập, có thể khai thác công dụng theo tính năng. 
B. Là vật trực tiếp phục vụ cho việc khai thác công dụng của vật chính, là một bộ phận của vật chính, nhưng có thể tách rời vật chính. 
C. Là sản vật tự nhiên mà tài sản mang lại. 
D. Là vật khi bị phân chia thì không giữ nguyên được tính chất và tính năng sử dụng ban đầu.
📑 Top 2 kết quả:
   01. (0.7604) Điều 110. Vật chính và vật phụ [KHOẢN] 1. Vật chính là vật độc lập, có thể khai thác công dụng theo tính năng. [KHOẢN] 2. Vật phụ là vật trực tiếp phục vụ cho việc khai thác công dụng của vật chính, l...
   02. (0.5856) Điều 111. Vật chia được và vật không chia được [KHOẢN] 1. Vật chia được là vật khi bị phân chia vẫn giữ nguyên tính chất và tính năng sử dụng ban đâu. [KHOẢN] 2. Vật không chia được là vật khi bị phân...

📄 File: 21-30.json — Câu 10
📘 Câu hỏi: Theo Bộ luật Dân sự, vật phụ là gì? 
A. Là vật độc lập, có thể khai thác công dụng theo tính năng. 
B. Là vật trực ti

⚖️ Đang chạy LegalRAGProcessor: 100%|██████████| 9/9 [28:47<00:00, 191.93s/it]


⏱️ Thời gian sinh trung bình mỗi câu: 18.80s

🎯 KẾT QUẢ TRUNG BÌNH:
Accuracy: 0.8000
Precision: 0.7993
Recall: 0.8000
F1: 0.7991

🔢 CONFUSION MATRIX:
        Pred A  Pred B  Pred C  Pred D
True A      11       2       1       1
True B       1      29       2       2
True C       2       3      15       1
True D       1       1       1      17

📁 File tổng hợp được lưu tại: results_json/rag_pipeline_results.json


In [13]:
!zip -r results_rag_llama_json.zip /kaggle/working/results_json

  adding: kaggle/working/results_json/ (stored 0%)
  adding: kaggle/working/results_json/41-50_q2_result.json (deflated 64%)
  adding: kaggle/working/results_json/81-90_q9_result.json (deflated 61%)
  adding: kaggle/working/results_json/31-40_q6_result.json (deflated 47%)
  adding: kaggle/working/results_json/71-80_q2_result.json (deflated 53%)
  adding: kaggle/working/results_json/61-70_q10_result.json (deflated 64%)
  adding: kaggle/working/results_json/81-90_q10_result.json (deflated 68%)
  adding: kaggle/working/results_json/31-40_q2_result.json (deflated 61%)
  adding: kaggle/working/results_json/71-80_q4_result.json (deflated 57%)
  adding: kaggle/working/results_json/21-30_q6_result.json (deflated 54%)
  adding: kaggle/working/results_json/61-70_q6_result.json (deflated 60%)
  adding: kaggle/working/results_json/81-90_q5_result.json (deflated 60%)
  adding: kaggle/working/results_json/51-60_q2_result.json (deflated 52%)
  adding: kaggle/working/results_json/21-30_q5_result.json 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
import gc, torch
from transformers import AutoTokenizer, AutoModelForCausalLM

class Decoder:
    def __init__(self, model_id="/kaggle/input/llama_law_tn_v3/keras/default/6"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16,   # ⚠️ FP16 tiết kiệm VRAM hơn BF16
            device_map="auto",
            low_cpu_mem_usage=True
        ).eval()
        torch.set_grad_enabled(False)

    def _build_context(self, question, retrieved_context):
        return """\
Bạn là **trợ lý pháp lý chuyên nghiệp**, chuyên tư vấn theo **pháp luật Việt Nam**.

⚖️ Nhiệm vụ:
Phân tích câu hỏi trắc nghiệm pháp luật và **chọn đáp án đúng nhất (A, B, C hoặc D)**.  
Chỉ trả lời **bằng tiếng Việt**, theo 2 phần bắt buộc:

1. **Đáp án đúng:** Ghi theo dạng "ĐÁP ÁN: B".
2. **Giải thích ngắn gọn:** Lý do vì sao lựa chọn đáp án này (không phân tích các phương án còn lại).

---

**Câu hỏi:**  
{query}

**Căn cứ pháp lý trích xuất:**  
{context_text}

---

➡️ **Phần trả lời chính thức:**  
"""


    def _generate_one(self, inputs, temperature, max_new_tokens):
        with torch.inference_mode():
            output = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=0.9,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )
        gen_ids = output[0][inputs["input_ids"].shape[-1]:]
        text = self.tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
        del output
        torch.cuda.empty_cache()
        gc.collect()
        return text

    def decode(self, prompt, temperature=0.3, max_new_tokens=256, num_samples=3):
        """
        ✅ Sinh tuần tự để tránh OOM (mỗi lần generate 1 câu).
        """
        results = []
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        for i in range(num_samples):
            text = self._generate_one(inputs, temperature, max_new_tokens)
            print(f"\n🧠 Hướng suy nghĩ #{i+1}:")
            print(text[:800], "..." if len(text) > 800 else "")
            results.append(text)
            torch.cuda.empty_cache()
        return results


In [15]:
import gc
import re
import time
import torch
from collections import Counter
from sentence_transformers import SentenceTransformer, util
import unicodedata
_RETRIEVAL_CACHE = {}


def _normalize_text_for_matching(text: str) -> str:
    """Normalize invisible chars, NBSP, various dashes, collapse spaces, NFC."""
    if not isinstance(text, str):
        return text
    # Unicode normalize
    text = unicodedata.normalize("NFC", text)
    # Replace common invisible / non-breaking spaces with normal space
    text = text.replace("\u00A0", " ").replace("\u200B", "").replace("\uFEFF", "")
    # Replace various dash types with simple hyphen
    for ch in ["\u2013", "\u2014", "\u2212", "–", "—", "−"]:
        text = text.replace(ch, "-")
    # Normalize multiple whitespace to single space
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def extract_choice_from_answer_strict(text: str):

    if not text or not isinstance(text, str):
        return None

    txt = _normalize_text_for_matching(text)

    patterns = [
        # e.g. "Đáp án đúng là: B - ...", "Đáp án: B", "Đáp án đúng là B"
        r'Đáp\s*án(?:\s+đúng)?\s*(?:là|:)?\s*([A-Da-d])(?=[\s\-\:\.\)]|$)',
        # line starting with "B. ..." or "B) ..."
        r'^\s*([A-Da-d])[\.\)\-]\s+',
        # "Chọn B"
        r'Chọn\s+([A-Da-d])\b',
        # short forms like "Đáp án B" (no colon)
        r'Đáp\s*án\s*([A-Da-d])\b',
    ]

    for pat in patterns:
        m = re.search(pat, txt, re.I | re.M)
        if m:
            g = m.group(1)
            if g and re.fullmatch(r'[A-Da-d]', g.strip()):
                return g.strip().upper()

    fallback_anchor_pat = r'(Đáp\s*án(?:\s+đúng)?)'
    m_anchor = re.search(fallback_anchor_pat, txt, re.I)
    if m_anchor:
        start = m_anchor.end()
        window = txt[start:start + 60]  # 60 chars window after 'Đáp án...'
        m2 = re.search(r'([A-Da-d])', window)
        if m2:
            return m2.group(1).upper()


    return None


def _retrieve_with_cache(retriever, query, top_k=5, use_cache=True):
    key = (query, top_k)
    if use_cache and key in _RETRIEVAL_CACHE:
        return _RETRIEVAL_CACHE[key]
    retrieved = retriever.retrieve(query, top_k=top_k)
    normalized = []
    for item in retrieved:
        try:
            if isinstance(item, (list, tuple)) and len(item) >= 1:
                normalized.append((item[0], item[1] if len(item) > 1 else None))
            else:
                normalized.append((str(item), None))
        except Exception:
            normalized.append((str(item), None))
    if use_cache:
        _RETRIEVAL_CACHE[key] = normalized
    return normalized


def _parse_choices_from_question(question_text):
    """Trả về dict {'A': textA, 'B': textB, ...} nếu tìm thấy A./B./..."""
    choices = {}
    lines = question_text.splitlines()
    pattern = re.compile(r'^\s*([A-Z])[\.\)]\s*(.+)', re.U)
    current_key = None
    for line in lines:
        m = pattern.match(line)
        if m:
            current_key = m.group(1).strip()
            choices[current_key] = m.group(2).strip()
        else:
            if current_key is not None and line.strip():
                choices[current_key] += " " + line.strip()
    if not choices:
        parts = re.split(r'\b([A-Z])[\.\)]\s+', question_text)
        for i in range(1, len(parts) - 1, 2):
            key = parts[i].strip()
            val = parts[i + 1].strip()
            choices[key] = val
    return choices
def shorten_final_answer(full_text: str, choice: str):
    """
    Trả về:
    - Câu chứa đáp án (A/B/C/D)
    - + câu giải thích ngắn nhất có chứa căn cứ pháp lý (Điều/Khoản/Luật/...).

    Nếu không tìm được câu pháp lý → chỉ lấy câu đáp án.
    """
    # Tách câu
    sents = re.split(r'(?<=[.?!])\s+', full_text.strip())

    # Regex xác định câu pháp lý
    law_keywords = r"(Điều|Khoản|Điểm|BLDS|BLHS|Bộ luật|Luật|Nghị định|Thông tư|Quyết định)\b"

    answer_sent = None
    law_sents = []

    # Tìm câu chứa đáp án
    for s in sents:
        if choice and re.search(rf'\b{choice}\b', s):
            answer_sent = s.strip()
            break  # ưu tiên câu đầu tiên chứa đáp án

    # Tìm câu chứa căn cứ pháp lý
    for s in sents:
        if re.search(law_keywords, s, re.I):
            law_sents.append(s.strip())

    # Nếu không có câu chứa đáp án -> fallback: lấy câu đầu dài nhất có nghĩa
    if not answer_sent:
        for s in sents:
            if len(s.strip()) > 20:
                answer_sent = s.strip()
                break
        if not answer_sent:
            return full_text.strip()

    # Nếu có câu pháp lý: chọn câu ngắn nhất để tránh dài dòng
    if law_sents:
        law_sent = sorted(law_sents, key=len)[0]
        # Trả về ghép 2 câu gọn nhất
        return f"{answer_sent} {law_sent}"

    # Nếu không có câu pháp lý → chỉ trả câu đáp án
    return answer_sent

def _has_law_keyword(text: str):
    return bool(re.search(r'\b(Điều|Khoản|Điểm|Bộ luật|Luật|Nghị định|Thông tư|Quyết định)\b', text, re.I))


def _has_explicit_answer_phrase(text: str):
    return bool(re.search(r'Đáp án(?:\s+đúng)?\s*(?:là|:)\s*[A-D]\b', text, re.I))


def legal_self_consistency_rag(
    retriever,
    decoder,
    query,
    num_samples: int = 3,
    temperature: float = 0.3,
    max_new_tokens: int = 384,
    top_k: int = 3,
    use_retrieval_cache: bool = True,
    dedup_threshold: float = 0.85,
):

    t0 = time.perf_counter()

    # 1) Retrieval (single)
    retrieved = _retrieve_with_cache(retriever, query, top_k=top_k, use_cache=use_retrieval_cache)
    context_text = "\n\n".join([doc for doc, _ in retrieved])

    # build prompt(block) as before
    choices = _parse_choices_from_question(query)
    choices_block = ""
    if choices:
        for k in sorted(choices.keys()):
            choices_block += f"{k}. {choices[k]}\n"
    else:
        choices_block = "(Không tìm thấy lựa chọn rõ ràng trong question.)\n"

    prompt = (
        "Dưới đây là một câu hỏi trắc nghiệm pháp lý. Dựa vào kiến thức pháp luật và ngữ cảnh cung cấp, "
        "hãy chọn 1 đáp án đúng nhất, không chọn quá 1 đáp án.\n\n"
        f"Câu hỏi:\n{query}\n\n"
        "Các lựa chọn:\n" + choices_block + "\n"
        "Ngữ cảnh pháp lý hỗ trợ:\n" + context_text + "\n\n"
        'Hãy phân tích ngắn gọn rồi trả lời ở định dạng: "Đáp án đúng là: <chữ cái> - <giải thích ngắn>".'
    )

    # 2) Sample outputs
    outputs = []
    print("\n🧠 Đang sinh các hướng lập luận:")
    for i in range(num_samples):
        try:
            out = decoder.decode(
                prompt,
                temperature=temperature,
                max_new_tokens=max_new_tokens,
                num_samples=1,
            )[0]
            outputs.append(out.strip())
            print(f"✅ Hướng {i+1} hoàn thành.")
        except Exception as e:
            print(f"⚠️ Lỗi khi sinh hướng {i+1}: {e}")
        finally:
            torch.cuda.empty_cache()
            gc.collect()

    if not outputs:
        print("⚠️ Không có câu trả lời nào được sinh.")
        return None

    # 3) Strict extraction of choices (do NOT modify outputs before extraction)
    extracted_choices = [extract_choice_from_answer_strict(o) for o in outputs]
    # keep parallel lists of (output, extracted_choice)
    paired = list(zip(outputs, extracted_choices))

    # Count only non-None choices
    counts = Counter([c for c in extracted_choices if c is not None])
    print("🧮 Tần suất lựa chọn (strict):", dict(counts))

    predicted_choice = None
    final_candidate = None

    if counts:
        # 4) Majority vote
        top_choice, top_count = counts.most_common(1)[0]
        total_votes = sum(counts.values())
        vote_ratio = top_count / total_votes

        # If clear majority (>50%) choose it
        if vote_ratio > 0.5:
            predicted_choice = top_choice
        else:
            # Tie / no clear majority: tie-break among choices
            # gather tied choices (those with max count)
            max_count = top_count
            tied_choices = [c for c, cnt in counts.items() if cnt == max_count]

            # tie-break heuristic 1: which tied choice has more candidates containing law keywords
            law_counts = {}
            explicit_counts = {}
            for c in tied_choices:
                law_counts[c] = 0
                explicit_counts[c] = 0
            for out, c in paired:
                if c in tied_choices:
                    if _has_law_keyword(out):
                        law_counts[c] += 1
                    if _has_explicit_answer_phrase(out):
                        explicit_counts[c] += 1

            # prefer higher law_counts, then explicit_counts, then first-seen
            # sort tied_choices by (law_counts desc, explicit_counts desc, index)
            first_seen = {c: next(i for i, pair in enumerate(paired) if pair[1] == c) for c in tied_choices}
            tied_choices_sorted = sorted(
                tied_choices,
                key=lambda x: (-law_counts.get(x, 0), -explicit_counts.get(x, 0), first_seen.get(x, 9999))
            )
            predicted_choice = tied_choices_sorted[0]

    else:
        # No choice extracted at all: fallback to pattern search across outputs for explicit "Đáp án: X"
        for out in outputs:
            c = extract_choice_from_answer_strict(out)
            if c:
                predicted_choice = c
                break

    # If still None, give up (no reliable predicted_choice)
    if predicted_choice is None:
        # fallback: return first output but mark no predicted_choice
        final_candidate = outputs[0]
        predicted_choice = None
        final_answer = shorten_final_answer(final_candidate, predicted_choice)
        return {
            "query": query,
            "choices": choices,
            "predicted_choice": predicted_choice,
            "predicted_answer": final_answer,
            "choice_similarity_scores": {},
            "all_candidates": outputs,
            "context_used": context_text,
            "timings": {"total_s": time.perf_counter() - t0},
        }

    # 5) Choose final candidate from outputs that declared predicted_choice (do NOT join)
    majority_indices = [i for i, c in enumerate(extracted_choices) if c == predicted_choice]
    majority_outputs = [outputs[i] for i in majority_indices]

    # Selection heuristics among majority_outputs (no embeddings):
    # priority: explicit 'Đáp án' phrase > contains law keyword > shortest length
    def candidate_score(text):
        s = 0
        if _has_explicit_answer_phrase(text):
            s += 100
        if _has_law_keyword(text):
            s += 20
        # shorter is slightly preferred (avoid verbose hallucination)
        s += max(0, 50 - len(text.split()))
        return s

    scored = [(candidate_score(out), out) for out in majority_outputs]
    scored.sort(key=lambda x: (-x[0], len(x[1])))  # higher score -> better, tie-breaker shorter
    final_candidate = scored[0][1]

    # 6) Shorten final candidate but preserve explicit answer token if present
    # ensure we don't remove the "Đáp án" token: pass original final_candidate to shorten_final_answer
    final_answer = shorten_final_answer(final_candidate, predicted_choice)

    t_total = time.perf_counter() - t0
    print(f"\n🏁 Tổng thời gian: {t_total:.1f}s")

    return {
        "query": query,
        "choices": choices,
        "predicted_choice": predicted_choice,
        "predicted_answer": final_answer,
        "choice_similarity_scores": {},   # not used to override
        "all_candidates": outputs,
        "context_used": context_text,
        "timings": {"total_s": t_total},
    }


In [16]:
decoder = Decoder('/kaggle/input/llama_law_tn_v3/keras/default/6')
result = legal_self_consistency_rag(retriever, decoder, question, num_samples = 3)
print('Câu trả lời cuối cùng:')
print(result["predicted_answer"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🔎 Truy vấn: Theo Bộ luật Dân sự, cá nhân có quyền có họ, tên (bao gồm cả chữ đệm, nếu có). Vậy, họ của cá nhân được xác định như thế nào trong trường hợp cha mẹ không có thỏa thuận? 
A. Theo họ của cha đẻ. 
B. Theo họ của cha đẻ hoặc họ của mẹ đẻ theo thỏa thuận của cha mẹ; nếu không có thỏa thuận thì họ của con được xác định theo tập quán. 
C. Theo họ của mẹ đẻ. 
D. Theo thỏa thuận của cha mẹ, hoặc nếu không có thỏa thuận thì theo quyết định của cơ quan nhà nước có thẩm quyền.
📑 Top 3 kết quả:
   01. (0.8936) Điều 26. Quyền có họ, tên [KHOẢN] 1. Cá nhân có quyền có họ, tên (bao gồm cả chữ đệm, nếu có). Họ, tên của một người được xác định theo họ, tên khai sinh của người đó. [KHOẢN] 2. Họ của cá nhân được x...
   02. (0.7196) Điều 28. Quyền thay đổi tên [KHOẢN] 1. Cá nhân có quyền yêu cầu cơ quan nhà nước có thẩm quyền công nhận việc thay đổi tên trong trường hợp sau đây: a) Theo yêu cầu của người có tên mà việc sử dụng tê...
   03. (0.6570) Điều 29. Quyền xác định, xác định lại dân tộ

In [19]:
import re
import json
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import os, json, time, gc, torch
from tqdm import tqdm


# ===========================================
# 🔹 Lấy A/B/C/D trong MCQ
# ===========================================
def extract_choice(text):
    """
    Trích xuất đáp án A/B/C/D từ text với nhiều pattern phong phú
    Ưu tiên: phần đầu văn bản > các pattern cụ thể > fallback
    """
    if not text or not isinstance(text, str):
        return None

    txt = text.strip()
    
    # Chỉ xét 400 ký tự đầu tiên (tăng từ 300 để bắt được nhiều pattern hơn)
    txt_head = txt[:400]

    # ========================================
    # 🎯 PRIORITY 1: Các pattern "Đáp án..."
    # ========================================
    priority_patterns = [
        # "Đáp án đúng là: B", "Đáp án là B", "Đáp án: B"
        r'Đáp\s*án(?:\s+đúng)?(?:\s+chính\s*xác)?(?:\s+cuối(?:\s+cùng)?)?(?:\s*là)?(?:\s*:)?\s*([A-Da-d])\b',
        
        # "Đáp án đúng: B -", "Đáp án B -"
        r'Đáp\s*án(?:\s+đúng)?\s*:\s*([A-Da-d])\s*[-\.]',
        
        # "Đáp án: B. Theo Điều..."
        r'Đáp\s*án\s*:\s*([A-Da-d])\.\s+',
    ]

    # ========================================
    # 🎯 PRIORITY 2: Các cụm từ đặc biệt
    # ========================================
    special_patterns = [
        # "B là đáp án chính xác"
        r'\b([A-Da-d])\s+là\s+đáp\s*án(?:\s+chính\s*xác|\s+đúng)',
        
        # "B là đáp án cuối"
        r'\b([A-Da-d])\s+là\s+đáp\s*án\s+cuối(?:\s+cùng)?',
        
        # "B là câu trả lời cuối cùng"
        r'\b([A-Da-d])\s+là\s+câu\s+trả\s+lời(?:\s+cuối(?:\s+cùng)?)?',
        
        # "Chọn B", "Chọn đáp án B"
        r'Chọn(?:\s+đáp\s*án)?\s+([A-Da-d])\b',
        
        # "Câu trả lời là B"
        r'Câu\s+trả\s+lời(?:\s+đúng)?(?:\s+là)?(?:\s*:)?\s*([A-Da-d])\b',
        
        # "Kết quả: B", "Kết luận: B"
        r'(?:Kết\s+quả|Kết\s+luận)(?:\s*là)?(?:\s*:)?\s*([A-Da-d])\b',
        
        # "Phương án B", "Lựa chọn B"
        r'(?:Phương\s+án|Lựa\s+chọn)\s+([A-Da-d])\b',
    ]

    # ========================================
    # 🔍 Tìm kiếm theo thứ tự ưu tiên
    # ========================================
    
    # STEP 1: Tìm trong priority patterns
    for pat in priority_patterns:
        m = re.search(pat, txt_head, re.I)
        if m:
            choice = m.group(1).upper()
            if choice in ['A', 'B', 'C', 'D']:
                return choice
    
    # STEP 2: Tìm trong special patterns
    for pat in special_patterns:
        m = re.search(pat, txt_head, re.I)
        if m:
            choice = m.group(1).upper()
            if choice in ['A', 'B', 'C', 'D']:
                return choice

    # ========================================
    # 🎯 FALLBACK 1: Pattern dạng "B. Text" hoặc "B) Text"
    # ========================================
    m = re.search(r'^\s*([A-Da-d])[\.\)\-:]\s+', txt_head, re.M)
    if m:
        choice = m.group(1).upper()
        if choice in ['A', 'B', 'C', 'D']:
            return choice

    # ========================================
    # 🎯 FALLBACK 2: Tìm chữ cái A-D đứng một mình
    # ========================================
    m = re.search(r'\b([A-D])\b', txt_head.upper())
    if m:
        return m.group(1)

    return None

# ===========================================
# 🔹 Evaluate cho Self-Consistency RAG
# ===========================================
def evaluate_multiple_choice(references, predictions):
    """
    references  : ground-truth choices
    predictions : model's final answers (string)
    """
    true_labels = []
    pred_labels = []

    for ref, pred in zip(references, predictions):
        true_labels.append(extract_choice(ref) or "X")
        pred_labels.append(extract_choice(pred) or "X")

    labels = ["A", "B", "C", "D", "X"]

    accuracy = accuracy_score(true_labels, pred_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels, pred_labels, labels=labels, average="weighted", zero_division=0
    )

    metrics = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1": f1
    }

    cm = confusion_matrix(true_labels, pred_labels, labels=labels)
    cm_df = pd.DataFrame(
        cm,
        index=[f"True {l}" for l in labels],
        columns=[f"Pred {l}" for l in labels]
    )

    return metrics, cm_df



# ===========================================
# 🔹 RAG + Self Consistency
# ===========================================
def run_rag_self_consistency_pipeline(json_folder, retriever, decoder, output_dir="results_json",
                     num_samples=3, temperature=0.3, max_new_tokens=256, top_k=2):
    """
    Chạy RAG pipeline với self-consistency:
    - retriever: object có hàm retrieve(query, top_k)
    - decoder  : object có decode() và (nếu có) _build_context()
    """

    os.makedirs(output_dir, exist_ok=True)

    all_refs, all_preds, results = [], [], []
    json_files = [f for f in os.listdir(json_folder) if f.endswith(".json")]

    # ⭐ NEW — biến đo thời gian trung bình
    total_time = 0.0
    total_questions = 0

    for filename in tqdm(json_files, desc="⚖️ Đang chạy LegalRAG Self-Consistency"):
        filepath = os.path.join(json_folder, filename)

        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)

        entries = data if isinstance(data, list) else [data]

        for idx, item in enumerate(entries):

            q = item.get("question", "")
            ref_choice = item.get("choice", "")
            context = item.get("context", "")
            pdf_file = item.get("pdf", "")

            if not q:
                continue

            try:
                start = time.time()

                # ⭐ Self-consistency RAG
                answer_dict = legal_self_consistency_rag(
                            retriever=retriever,
                            decoder=decoder,
                            query=q,
                            num_samples=num_samples,
                            temperature=temperature,
                            max_new_tokens=max_new_tokens,
                            top_k=top_k
                            )

                # Lấy output từ dict
                predicted_answer = answer_dict.get("predicted_answer", "")
                predicted_choice = answer_dict.get("predicted_choice", "")
                candidates = answer_dict.get("all_candidates", [])
                choices = answer_dict.get("choices", [])
                choice_similarity_scores = answer_dict.get("choice_similarity_scores", {})
                context_used = answer_dict.get("context_used", "")
                timings = answer_dict.get("timings", {})
                elapsed = time.time() - start

                # ⭐ NEW — cộng dồn thời gian
                total_time += elapsed
                total_questions += 1

                # ==========================
                # In ra console
                # ==========================
                print("\n" + "=" * 80)
                print(f"📄 File: {filename} — Câu {idx+1}")
                print(f"📘 Câu hỏi: {q}")
                print(f"💬 Final Answer: {predicted_answer}")
                print(f"🔤 Predicted Choice: {predicted_choice}")
                print(f"🗳️ All candidates ({len(candidates)}):")
                for c_idx, c in enumerate(candidates):
                    print(f"   - [{c_idx}] {c}")
                print(f"⏱️ Time: {elapsed}s")
                print("=" * 80)

                # ==========================
                # Lưu để đánh giá MCQ
                # ==========================
                all_refs.append(ref_choice)
                all_preds.append(predicted_choice)

                # ==========================
                # Lưu full result
                # ==========================
                sample_result = {
                    "id": item.get("id", None),
                    "file": filename,
                    "question": q,
                    "ground_truth_choice": ref_choice,
                    "predicted_choice": predicted_choice,
                    "predicted_answer": predicted_answer,
                    "choices_text": choices,
                    "choice_similarity_scores": choice_similarity_scores,
                    "all_candidates": candidates,
                    "context_used": context_used,
                    "timings": timings,
                    "context_raw": context,
                    "pdf": pdf_file,
                }

                results.append(sample_result)

                # Lưu từng câu
                base = os.path.splitext(filename)[0]
                out_path = os.path.join(output_dir, f"{base}_q{idx+1}_result.json")
                with open(out_path, "w", encoding="utf-8") as fw:
                    json.dump(sample_result, fw, ensure_ascii=False, indent=2)

                torch.cuda.empty_cache()
                gc.collect()

            except Exception as e:
                print(f"❌ Lỗi ở {filename} (câu {idx}): {repr(e)}")
                continue

    # ==========================================================
    # 🔥 ĐÁNH GIÁ MCQ
    # ==========================================================
    if not all_refs:
        print("⚠️ Không có kết quả để đánh giá.")
        return {}, []

    metrics, cm_df = evaluate_multiple_choice(all_refs, all_preds)

    print("\n🎯 KẾT QUẢ TRUNG BÌNH:")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

    print("\n🔢 CONFUSION MATRIX:")
    print(cm_df)

    out_path = os.path.join(output_dir, "rag_pipeline_results.json")
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump({"results": results, "overall_metrics": metrics}, f, ensure_ascii=False, indent=2)

    cm_df.to_csv(os.path.join(output_dir, "confusion_matrix.csv"), encoding="utf-8-sig")

    # ⭐ NEW — in thời gian trung bình mỗi câu
    if total_questions > 0:
        avg_time = total_time / total_questions
        print(f"\n⏱️ Thời gian trung bình mỗi câu: {avg_time:.4f} giây")

    print(f"\n📁 Lưu xong: {out_path}")

    return metrics, results

In [20]:
json_folder = '/kaggle/input/luat-benchmark-tn/Benchmark_QA'
predictions, results = run_rag_self_consistency_pipeline(json_folder, retriever, decoder, output_dir="rag_selfconsistency_pipeline_results.json")

⚖️ Đang chạy LegalRAG Self-Consistency:   0%|          | 0/9 [00:00<?, ?it/s]

🔎 Truy vấn: Người sử dụng lao động có quyền tạm đình chỉ công việc của người lao động trong thời hạn tối đa là bao lâu trong trường hợp bình thường? 
A. Không quá 07 ngày. 
B. Không quá 15 ngày. 
C. Không quá 30 ngày. 
D. Không quá 90 ngày.
📑 Top 2 kết quả:
   01. (0.9993) Điều 128. Tạm đình chỉ công việc [KHOẢN] 1. Người sử dụng lao động có quyền tạm đình chỉ công việc của người lao động khi vụ việc vi phạm có những tình tiết phức tạp nếu xét thấy để người lao động tiế...
   02. (0.9967) Điều 31. Nhận lại người lao động hết thời hạn tạm hoãn thực hiện hợp đồng lao động | Trong thời hạn 15 ngày kế từ ngày hết thời hạn tạm hoãn thực hiện hợp đồng lao động, người lao động phải có mặt tại...

🧠 Đang sinh các hướng lập luận:

🧠 Hướng suy nghĩ #1:
Đáp án đúng là B. Theo Điều 128 Khoản 2 Bộ luật Lao động, thời hạn tạm đình chỉ công việc không được quá 15 ngày, trường hợp đặc biệt không được quá 20 ngày. [KHOẢN] 2. Thời hạn tạm đình chỉ công việc không được quá 15 ngày, trường hợp đặc biệt kh

⚖️ Đang chạy LegalRAG Self-Consistency:  11%|█         | 1/9 [09:51<1:18:52, 591.55s/it]

🔎 Truy vấn: Thời gian nghỉ giữa giờ đối với người lao động làm việc theo thời giờ làm việc bình thường từ 06 giờ trở lên trong một ngày là bao lâu? 
A. Ít nhất 30 phút liên tục. 
B. Ít nhất 45 phút liên tục. 
C. Ít nhất 60 phút liên tục. 
D. Do hai bên thỏa thuận.
📑 Top 2 kết quả:
   01. (0.9969) Điều 105. Thời giờ làm việc bình thường [KHOẢN] 1. Thời giờ làm việc bình thường không quá 08 giờ trong 01 ngày và không quá 48 giờ trong 01 tuần. [KHOẢN] 2. Người sử dụng lao động có quyền quy định t...
   02. (0.9935) Điều 105 của Bộ luật này từ 06 giờ trở lên trong một ngày thì được nghỉ giữa giờ Ít nhất 30 phút liên tục, làm việc ban đêm thì được nghỉ giữa giờ ít nhất 45 phút liên tục. Trường hợp người lao động l...

🧠 Đang sinh các hướng lập luận:

🧠 Hướng suy nghĩ #1:
Đáp án đúng là A. Theo Điều 105 Khoản 3 Bộ luật Lao động, khi làm việc từ 06 giờ trở lên trong một ngày thì được nghỉ giữa giờ ít nhất 30 phút liên tục. [KHOẢN] 2. Ngoài thời gian nghỉ quy định tại khoản Ì Điều này, người s

⚖️ Đang chạy LegalRAG Self-Consistency:  22%|██▏       | 2/9 [19:44<1:09:06, 592.31s/it]

...

🔎 Truy vấn: Thỏa thuận về chế độ tài sản của vợ chồng phải được lập vào thời điểm nào? 
A. Trong thời kỳ hôn nhân. 
B. Sau khi đăng ký kết hôn. 
C. Bất cứ lúc nào trong thời kỳ hôn nhân. 
D. Trước khi kết hôn.
📑 Top 2 kết quả:
   01. (0.9996) Điều 47. Thỏa thuận xác lập chế độ tài sản của vợ chồng Trong trường hợp hai bên kết hôn lựa chọn chế độ tài sản theo thỏa thuận thì thỏa thuận này phải được lập trước khi kết hôn, bằng hình thức văn ...
   02. (0.9988) Điều 64 của Luật này. Chế độ tài sản của vợ chồng theo thỏa thuận được thực hiện theo quy định tại các điều 47, 48, 49, 50 và 59 của Luật này. [KHOẢN] 2. Các quy định tại các điều 29, 30, 31 và 32 của...

🧠 Đang sinh các hướng lập luận:

🧠 Hướng suy nghĩ #1:
Lưu ý: Đáp án đúng là D. Theo Điều 47 của Luật, thỏa thuận về chế độ tài sản của vợ chồng phải được lập trước khi kết hôn. 
Đáp án đúng là D. Theo Điều 47 của Luật, thỏa thuận này phải được lập trước khi kết hôn, bằng hình thức văn bản có công chứng hoặc chứng thực. [ĐIỀU LUẬT]

⚖️ Đang chạy LegalRAG Self-Consistency:  89%|████████▉ | 8/9 [1:18:32<09:47, 587.19s/it]

🔎 Truy vấn: Điều kiện để một tổ chức được công nhận là pháp nhân là gì? 
A. Được thành lập theo quy định của luật và có tài sản độc lập. 
B. Có cơ cấu tổ chức theo quy định tại Điều 83 của Bộ luật này. 
C. Nhân danh mình tham gia quan hệ pháp luật một cách độc lập. 
D. Cả A, B và C.
📑 Top 2 kết quả:
   01. (0.9979) Điều 74. Pháp nhân [KHOẢN] 1. Một tổ chức được công nhận là pháp nhân khi có đủ các điều kiện sau đây: a) Được thành lập theo quy định của Bộ luật này, luật khác có liên quan; b) Có cơ cấu tổ chức the...
   02. (0.9619) Điều 83 của Bộ luật này; _ ©) Có tài sản độc lập với cá nhân, pháp nhân khác và tự chịu trách nhiệm băng tài sản của mình; đ) Nhân danh mình tham gia quan hệ pháp luật một cách độc lập. [KHOẢN] 2. Mọi...

🧠 Đang sinh các hướng lập luận:

🧠 Hướng suy nghĩ #1:
Lưu ý: Đáp án đúng có thể là A, B, C hoặc D, tùy thuộc vào đáp án đúng nhất theo Điều 74 Bộ luật Dân sự. 
Trường hợp đáp án đúng là D thì phải giải thích như sau: Đáp án đúng là D. Điều 74 Khoản 1 Bộ luật

⚖️ Đang chạy LegalRAG Self-Consistency: 100%|██████████| 9/9 [1:28:10<00:00, 587.83s/it]


🎯 KẾT QUẢ TRUNG BÌNH:
Accuracy: 0.7333
Precision: 0.7545
Recall: 0.7333
F1: 0.7248

🔢 CONFUSION MATRIX:
        Pred A  Pred B  Pred C  Pred D  Pred X
True A      10       4       0       1       0
True B       3      29       1       1       0
True C       1       5      10       5       0
True D       1       2       0      17       0
True X       0       0       0       0       0

⏱️ Thời gian trung bình mỗi câu: 58.4000 giây

📁 Lưu xong: rag_selfconsistency_pipeline_results.json/rag_pipeline_results.json


In [21]:
!zip -r results_self_consistency_llama_json.zip /kaggle/working/rag_selfconsistency_pipeline_results.json

  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/ (stored 0%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/41-50_q2_result.json (deflated 80%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/81-90_q9_result.json (deflated 78%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/31-40_q6_result.json (deflated 79%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/71-80_q2_result.json (deflated 78%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/61-70_q10_result.json (deflated 79%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/81-90_q10_result.json (deflated 83%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/31-40_q2_result.json (deflated 81%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/71-80_q4_result.json (deflated 76%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/21-30_q6_result.json (defla

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
